In [160]:
import pandas as pd
data=pd.read_csv('Placement_Data_Full_Class.csv',encoding="utf-8")

In [161]:
data = data.drop(['sl_no'], axis=1)
data = data.drop(['salary'], axis=1)
# y = data['status']
# y = pd.Categorical(data['status'].map({'Placed': 1, 'Not Placed': 0}))
y = data['status'].map({'Placed': 1, 'Not Placed': 0})
X = data.drop(['status'], axis=1)
X = pd.get_dummies(X)

In [162]:
# X.head

In [163]:
# y

In [184]:
y.describe()

count    215.000000
mean       0.688372
std        0.464240
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: status, dtype: float64

In [165]:
# import pandas_profiling as pdp

In [166]:
# d = pd.read_csv('ice.csv')
# d = pd.read_csv('ice.csv', encoding='utf-8')
# d.profile_report()
# pdp.ProfileReport(data)
# from IPython.display import HTML
# report = pandas_profiling.ProfileReport(d)
# inner_html = report.to_html().replace('"', "&quot;")
# html = '<iframe srcdoc="{}" width=1200 height=1200>'.format(inner_html)
# display.HTML(html)

In [167]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# for i in data.columns.values.tolist():
#  if (i=='age'):
#   pass
#  else:
#   titanic[i] = le.fit_transform(titanic[i])

In [168]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=54,shuffle=True)

In [169]:
# from sklearn_extensions.extreme_learning_machines.elm import GenELMClassifier
# from sklearn_extensions.extreme_learning_machines.random_layer import RBFRandomLayer, MLPRandomLayer

In [170]:
# nh = 4
# srhl_tanh = MLPRandomLayer(n_hidden=nh, activation_func='tanh', random_state=0)
# srhl_rbf = RBFRandomLayer(n_hidden=nh*2, rbf_width=0.1, random_state=0)
# clf1=GenELMClassifier(hidden_layer=srhl_tanh)

In [171]:
# clf1.fit(X_train,y_train)
# print(clf1.score(X_test,y_test))

In [172]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'n_jobs':-1,
    'bagging_freq': 5,
    'verbose': 0
}

clf = lgb.train(params,
                lgb_train,
                num_boost_round=80,
                valid_sets=lgb_eval,
                verbose_eval=1,
                early_stopping_rounds=5)
y_pred = clf.predict(X_test, num_iteration=clf.best_iteration)
pred=[]
for i in y_pred:
 if i>0.5: pred.append(1)
 else:pred.append(0)
y_pred=pred
print(y_pred)
from sklearn.metrics import *
print("roc:%.4f"%roc_auc_score(y_test,y_pred))
print("acc:%.4f"%accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))
print("f1:%.4f"%f1_score(y_test, y_pred))
print('recall:','%.4f'%recall_score(y_test, y_pred))
print('precision:','%.4f'%precision_score(y_test, y_pred))

[1]	valid_0's binary_logloss: 0.578628
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.559844
[3]	valid_0's binary_logloss: 0.543054
[4]	valid_0's binary_logloss: 0.52943
[5]	valid_0's binary_logloss: 0.515642
[6]	valid_0's binary_logloss: 0.504116
[7]	valid_0's binary_logloss: 0.493722
[8]	valid_0's binary_logloss: 0.482091
[9]	valid_0's binary_logloss: 0.473021
[10]	valid_0's binary_logloss: 0.469547
[11]	valid_0's binary_logloss: 0.457898
[12]	valid_0's binary_logloss: 0.447274
[13]	valid_0's binary_logloss: 0.437579
[14]	valid_0's binary_logloss: 0.430359
[15]	valid_0's binary_logloss: 0.423389
[16]	valid_0's binary_logloss: 0.419231
[17]	valid_0's binary_logloss: 0.414649
[18]	valid_0's binary_logloss: 0.411238
[19]	valid_0's binary_logloss: 0.408202
[20]	valid_0's binary_logloss: 0.403474
[21]	valid_0's binary_logloss: 0.398603
[22]	valid_0's binary_logloss: 0.393462
[23]	valid_0's binary_logloss: 0.388895
[24]	valid_0's binary_logloss:

In [173]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=90)
X_train_resampled, y_train_resampled = smt.fit_sample(X_train, y_train)

In [174]:
# clf1.fit(X_train_resampled,y_train_resampled)
# print(clf1.score(X_test,y_test))

In [175]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train_resampled, y_train_resampled)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'n_jobs':-1,
    'bagging_freq': 5,
    'verbose': 0
}

clf = lgb.train(params,
                lgb_train,
                num_boost_round=80,
                valid_sets=lgb_eval,
                verbose_eval=1,
                early_stopping_rounds=5)
y_pred = clf.predict(X_test, num_iteration=clf.best_iteration)
pred=[]
for i in y_pred:
 if i>0.5: pred.append(1)
 else:pred.append(0)
y_pred=pred
print(y_pred)
from sklearn.metrics import *
print("roc:%.4f"%roc_auc_score(y_test,y_pred))
print("acc:%.4f"%accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))
print("f1:%.4f"%f1_score(y_test, y_pred))
print('recall:','%.4f'%recall_score(y_test, y_pred))
print('precision:','%.4f'%precision_score(y_test, y_pred))

[1]	valid_0's binary_logloss: 0.672922
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.655537
[3]	valid_0's binary_logloss: 0.638689
[4]	valid_0's binary_logloss: 0.623661
[5]	valid_0's binary_logloss: 0.6104
[6]	valid_0's binary_logloss: 0.601301
[7]	valid_0's binary_logloss: 0.587418
[8]	valid_0's binary_logloss: 0.575047
[9]	valid_0's binary_logloss: 0.560795
[10]	valid_0's binary_logloss: 0.55504
[11]	valid_0's binary_logloss: 0.544186
[12]	valid_0's binary_logloss: 0.53444
[13]	valid_0's binary_logloss: 0.523204
[14]	valid_0's binary_logloss: 0.51293
[15]	valid_0's binary_logloss: 0.506214
[16]	valid_0's binary_logloss: 0.498946
[17]	valid_0's binary_logloss: 0.493817
[18]	valid_0's binary_logloss: 0.489875
[19]	valid_0's binary_logloss: 0.48065
[20]	valid_0's binary_logloss: 0.475793
[21]	valid_0's binary_logloss: 0.46967
[22]	valid_0's binary_logloss: 0.46421
[23]	valid_0's binary_logloss: 0.461472
[24]	valid_0's binary_logloss: 0.4515